In [23]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [24]:
faceCascade = cv2.CascadeClassifier('./haarcascades/haarcascade_frontalface_default.xml')
eyesCascade = cv2.CascadeClassifier('./haarcascades/haarcascade_eye.xml')

In [25]:
path_to_data = './images_dataset/'
path_to_cropped_data = './images_dataset/cropped'

In [26]:
img_dirs = []
for files in os.scandir(path_to_data):
    if files.is_dir():
        img_dirs.append(files.path)
img_dirs

['./images_dataset/cristiano_ronaldo',
 './images_dataset/lionel_messi',
 './images_dataset/roger_federer',
 './images_dataset/serena_williams',
 './images_dataset/virat_kohli']

In [27]:
import shutil
if os.path.exists(path_to_cropped_data):
    shutil.rmtree(path_to_cropped_data)
os.mkdir(path_to_cropped_data)

In [28]:
def get_cropped_img(img_path):
    img = cv2.imread(img_path)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray_img, 1.5, 5)
    for (x,y,w,h) in faces:
        roi_gray = gray_img[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eyesCascade.detectMultiScale(roi_gray)
        if len(eyes) >= 2:
            return roi_color

In [29]:
cropped_img_dirs = []
celebrity_file_dict = {}
for img_dir in img_dirs:
    count = 1
    celebrity_name = img_dir.split('/')[-1]
    celebrity_file_dict[celebrity_name] = []
    for entry in os.scandir(img_dir):
        roi_color = get_cropped_img(entry.path)
        if roi_color is not None:
            cropped_folder = path_to_cropped_data + '/' + celebrity_name
            if not os.path.exists(cropped_folder):
                os.mkdir(cropped_folder)
                cropped_img_dirs.append(cropped_folder)
                print("Generating cropped folder for: ", celebrity_name)
            cropped_file_name = celebrity_name + str(count) + '.png'
            cropped_file_path = cropped_folder + '/' + cropped_file_name
            cv2.imwrite(cropped_file_path, roi_color)
            celebrity_file_dict[celebrity_name].append(cropped_file_path)
            count+=1

Generating cropped folder for:  cristiano_ronaldo
Generating cropped folder for:  lionel_messi
Generating cropped folder for:  roger_federer
Generating cropped folder for:  serena_williams
Generating cropped folder for:  virat_kohli
